# Load the Dataset

In [3]:
import pandas as pd
#encode categorical and numerical columns with label and ordinal encoding
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
import numpy as np
#do random forest classification
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#import train_test_split
from sklearn.model_selection import train_test_split,PredefinedSplit


seed = 42
FILENAME = "train_dataset.csv"
# 1, 2 e 3
df = pd.read_csv(FILENAME, sep=",", low_memory=False)
df = df.dropna()
df = df.drop(columns=["label"])

# 4
df["src_bytes"] = df["src_bytes"].replace("0.0.0.0", np.nan).astype(float)
mean_src_bytes = df["src_bytes"].mean()
df["src_bytes"] = df["src_bytes"].fillna(mean_src_bytes)

# 5
df.astype({'src_bytes': 'int64', 'ts': 'datetime64[ms]', 'dns_AA': 'bool', 'dns_RD': 'bool', 'dns_RA': 'bool', 'dns_rejected': 'bool', 'ssl_resumed': 'bool', 'ssl_established': 'bool', 'weird_notice': 'bool'}).dtypes

# 6
y = df["type"]
df = df.drop(columns=["type"])

print(df.select_dtypes(include=['object']).shape)

# 7
oe = OrdinalEncoder()
df_oe = oe.fit_transform(df.select_dtypes(include=['object']))
df.loc[:, df.select_dtypes(include=['object']).columns] = df_oe
X = df.to_numpy()

# 8
le = LabelEncoder()
y = le.fit_transform(y)

indeces = np.arange(X.shape[0])
train_idx, val_idx = train_test_split(indeces, test_size=0.2, stratify=y, random_state=seed)


fold = np.zeros(X.shape[0])
fold[train_idx] = -1
fold[val_idx] = 0

ps = PredefinedSplit(fold)
print("train idx: ",train_idx)
print("val idx: ",val_idx)
print("n splits: ",ps.get_n_splits())

X_train = X[train_idx,:]
y_train = y[train_idx]
X_val = X[val_idx,:]
y_val = y[val_idx]

(617002, 27)
train idx:  [ 23754 272651 328894 ... 523435 430660 552899]
val idx:  [ 93259  45326 396437 ... 523991 398025 211949]
n splits:  1


In [4]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

X = scaler.transform(X)

pca = PCA(n_components='mle', svd_solver='full')
pca.fit(X_train)
X = pca.transform(X)

# Apply PCA transformation to and X_val
X_val = pca.transform(X_val)


# SVM

In [5]:
# from sklearn.svm import SVC
# from sklearn.model_selection import GridSearchCV
# import pickle

# scoring = ['balanced_accuracy','f1_weighted']

# param_grid = {'C': [0.1, 1,10,100,1000], 
#               'gamma': [1,0.1,0.01,0.001],
#               'kernel': ['rbf','sigmoid','linear']}  

# #cast x and y train in numpy arrays
# X_train = np.array(X_train)
# y_train = np.array(y_train)

# grid = GridSearchCV(SVC(random_state=seed), param_grid,verbose=10,cv=ps,n_jobs=-1,scoring=scoring)
# grid.fit(X_train, y_train)
# file = open("svc.save","wb")
# pickle.dump(grid.best_estimator_, file)
# print("Best hypearparameters: ",grid.best_estimator_)
# print("Best performance:",  grid.best_score_)
# file.close()



# Random Forest

In [6]:
# #import random forest classifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# import pickle

# param_grid = {  'n_estimators': [100, 200],
#                 'criterion': ['gini', 'entropy']
#                 }  

# scoring = ['balanced_accuracy','f1_weighted']

# grid = GridSearchCV(RandomForestClassifier(random_state=seed,n_jobs=-1), param_grid,verbose=10,cv=ps,n_jobs=-1,scoring=scoring, refit='balanced_accuracy')
# grid.fit(X, y)
# file = open("rf.save","wb")
# pickle.dump(grid.best_estimator_, file)
# print("Best hyperparameters: ",grid.best_estimator_)
# print("Best performance:",  grid.best_score_)
# file.close()


# KNN

In [7]:

#do the same with knn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import pickle
param_grid = {'n_neighbors': [15],
            'p': [1]}
scoring = ['balanced_accuracy','f1_weighted']
grid = GridSearchCV(KNeighborsClassifier(), param_grid,verbose=10,cv=ps,scoring=scoring, refit='balanced_accuracy')
grid.fit(X, y)
#save the best model
print("Best hyperparameters: ",grid.best_estimator_)
print("Best performance:",  grid.best_score_)
#save the model with the best hyperparameters
file = open("knn.save","wb")
pickle.dump(grid.best_estimator_, file)
file.close()

#do prediction on the validation set
y_pred = grid.predict(X_val)
cm = confusion_matrix(y_val, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

#drop balanced_accuracy and f1_weighted

#balanced_accuracy print
print("balanced_accuracy: ",grid.cv_results_['mean_test_balanced_accuracy'][grid.best_index_])
#f1_weighted print
print("f1_weighted: ",grid.cv_results_['mean_test_f1_weighted'][grid.best_index_])

Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV 1/1; 1/1] START n_neighbors=15, p=1.........................................


c:\Users\rinal\Desktop\UNIBO\DA\data-analytics\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\rinal\Desktop\UNIBO\DA\data-analytics\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[CV 1/1; 1/1] END n_neighbors=15, p=1; balanced_accuracy: (test=0.977) f1_weighted: (test=0.997) total time= 1.6min
Best hyperparameters:  KNeighborsClassifier(n_neighbors=15, p=1)
Best performance: 0.9771813550423756


NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.